# Tratamento e Limpeza de Dados
## Classificação de Intensidade de Chuva - Problema 7.8

**Objetivo:** Tratar valores faltantes, limpar dados e classificar a intensidade de chuva.

**Classificação:**
- Sem chuva: 0 mm
- Leve: 0.1 - 2.5 mm/h
- Moderada: 2.6 - 10 mm/h
- Forte: > 10 mm/h

---


## 1. Importação de Bibliotecas


In [1]:
import sys
import subprocess

def install_if_missing(package):
    try:
        __import__(package.split('-')[0].split('[')[0])
    except ImportError:
        print(f"Instalando {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "--quiet", package])

dependencies = ['pandas', 'numpy', 'psycopg2-binary']

for dep in dependencies:
    install_if_missing(dep)

import pandas as pd
import numpy as np
import psycopg2
from psycopg2.extras import execute_batch
import warnings
warnings.filterwarnings('ignore')

print("Bibliotecas importadas com sucesso!")


Instalando psycopg2-binary...
Bibliotecas importadas com sucesso!


## 2. Conexão e Carregamento


In [2]:
DB_CONFIG = {
    'host': 'postgres',
    'port': 5432,
    'database': 'inmet_db',
    'user': 'inmet_user',
    'password': 'inmet_password'
}

query = "SELECT * FROM dados_meteorologicos ORDER BY timestamp_utc"

try:
    conn = psycopg2.connect(**DB_CONFIG)
    conn.set_client_encoding('UTF8')
    df = pd.read_sql(query, conn)
    conn.close()
    df['timestamp_utc'] = pd.to_datetime(df['timestamp_utc'])
    print(f"Dados carregados: {len(df):,} registros")
except Exception as e:
    print(f"Erro: {e}")
    df = pd.DataFrame()

if len(df) > 0:
    print(f"Valores faltantes: {df.isnull().sum().sum()}")
    display(df.head())


Dados carregados: 177,380 registros
Valores faltantes: 1774281


,id,codigo_wmo,data,hora_utc,timestamp_utc,precipitacao_mm,pressao_estacao_mb,pressao_max_mb,pressao_min_mb,radiacao_global_kjm2,...,temperatura_orvalho_max_c,temperatura_orvalho_min_c,umidade_rel_max_pct,umidade_rel_min_pct,umidade_rel_horaria_pct,vento_direcao_graus,vento_rajada_max_ms,vento_velocidade_ms,intensidade_chuva,created_at
0,177380,A370,2024-01-01,00:00:00,2024-01-01,0.0,0.0,None,None,0.0,...,None,None,None,None,0.0,0.0,None,0.0,sem_chuva,2025-12-04 05:40:50.855141
1,16080,A307,2024-01-01,00:00:00,2024-01-01,0.0,970.5,None,None,0.0,...,None,None,None,None,67.0,294.0,None,2.2,sem_chuva,2025-12-04 05:40:08.694444
2,32160,A309,2024-01-01,00:00:00,2024-01-01,0.0,0.0,None,None,0.0,...,None,None,None,None,0.0,0.0,None,0.0,sem_chuva,2025-12-04 05:40:12.305180
3,48240,A322,2024-01-01,00:00:00,2024-01-01,0.0,922.1,None,None,0.0,...,None,None,None,None,0.0,64.0,None,3.5,sem_chuva,2025-12-04 05:40:15.787507
4,64320,A328,2024-01-01,00:00:00,2024-01-01,0.0,0.0,None,None,0.0,...,None,None,None,None,0.0,0.0,None,0.0,sem_chuva,2025-12-04 05:40:19.136318


## 3. Tratamento de Valores Faltantes


In [3]:
if len(df) > 0:
    # Precipitação nula = 0 (sem chuva)
    df['precipitacao_mm'] = df['precipitacao_mm'].fillna(0)
    
    # Outras variáveis: mediana por estação
    vars_numericas = ['pressao_estacao_mb', 'temperatura_ar_c', 'umidade_rel_horaria_pct', 
                      'vento_velocidade_ms', 'radiacao_global_kjm2']
    
    for var in vars_numericas:
        if var in df.columns:
            medianas = df.groupby('codigo_wmo')[var].transform('median')
            df[var] = df[var].fillna(medianas).fillna(df[var].median())
    
    print(f"✅ Valores faltantes após tratamento: {df.isnull().sum().sum()}")
else:
    print("⚠️  Nenhum dado disponível!")


✅ Valores faltantes após tratamento: 1773800


## 4. Classificação de Intensidade de Chuva


In [4]:
def classificar_intensidade_chuva(precipitacao):
    """Classifica conforme problema 7.8"""
    if pd.isna(precipitacao) or precipitacao == 0:
        return 'sem_chuva'
    elif 0.1 <= precipitacao <= 2.5:
        return 'leve'
    elif 2.6 <= precipitacao <= 10:
        return 'moderada'
    elif precipitacao > 10:
        return 'forte'
    else:
        return 'sem_chuva'

if len(df) > 0:
    df['intensidade_chuva'] = df['precipitacao_mm'].apply(classificar_intensidade_chuva)
    print("Distribuição de classes:")
    display(df['intensidade_chuva'].value_counts())


Distribuição de classes:


intensidade_chuva
sem_chuva    170552
leve           5945
moderada        759
forte           124
Name: count, dtype: int64

## 5. Atualização no Banco de Dados


In [5]:
if len(df) > 0:
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        cursor = conn.cursor()
        
        update_query = "UPDATE dados_meteorologicos SET intensidade_chuva = %s WHERE id = %s"
        dados_atualizar = [(row['intensidade_chuva'], row['id']) for _, row in df.iterrows()]
        execute_batch(cursor, update_query, dados_atualizar, page_size=1000)
        
        conn.commit()
        cursor.close()
        conn.close()
        
        print(f"{len(df):,} registros atualizados no banco!")
    except Exception as e:
        print(f"Erro: {e}")
else:
    print("Nenhum dado para atualizar!")


177,380 registros atualizados no banco!
